# Sequence Mining
* What is sequence mining? 
    * Helps discover **patterns** in a dataest.
* There are two types of patterns we can look for
    * sequences: allows gaps
    * substrings: do not allow gaps between elements

## 10.1 Frequent Sequences 
* $\sum$ is an *alphabet*: a finite set of characters
* sequence or string: **ordered** list of symbols
    * ordered doesn't imply sorted, just that order matters.
    * ($\ni$ means such that)
    * writen as $s = s_1, s_2, ..., s_k \text{ where } s_i \in \sum$ is a symbol at position i, also denoted as s[i]
* A sequence with size k is denoted as k-sequence.
* s[i:j] denotes a substring of consecutive symbols from i through j.
    * $s[i:j] = s_i, s_{i+1}, ..., s_{j-1}, s_j$ to denote substring.
* Prefix of a sequence s: first i letters
    * $s[1:i] = s_1s_2...s_i$ with 0 <= i <= n.
* suffix of a sequence s: last n-i letters
    * $s[i:n] = s_is_{i+1}...s_n$ with 1 <= i <= n.
* $\sum^{*}$ be the set of all possible sequences that can be constructed using the symbols in $\sum$
    * this includes the empty set
    * essentially the power set 
* $s=s_1s_2...s_n$ and $r = r_1r_2...r_m$ be two sequences over $\sum$.
    * r is a *subsequence* if $r \subseteq s$, IF there's a one to one mapping of characters between r and s for any two positions.
    * Each position in r is mappting to a different position in s **and the orders of symbols is preserved** even though there may be **gaps** between consecutive elements (non-contiguous).
    * if r is a subsequence of s, $r \subseteq s$ we say that s *contians* r.
        * this means tthat r can be either be a subsequence OR a consecutive subsequence of s.
    * sequence r is a *consecutive subsequence* or **substring** of s if there are no gaps between the elements of r in the mapping.
        * E.X.: $\sum = \{A, C, G, T\}$ and $s = ACTGAACG$
        * $r_1 = CGAAG$ is a subsequence of s.
        * $r_2 = CTGA$ is a substring of s
        * $r_3 = ACT$ is a prefix of s.
        * $s_r = ACTGA$ is also a prefix of s.
        * $r_5 = GAACG$ is one of the suffixes.
    * Given a database of N sequences or strings, given some sequence r, the *support* of r in the database is the total number of sequences in D that *contain* r, is a subsequence or a substring in s.
    * $sup(r) = |\{s_i \in D|r \subseteq s_i\}|$
    * relative support: $rsup(r) = sup(R)/N$
    * A sequence is frequent if sup(r) >= minsup.
    * A frequent sequence is *maximal* if it's not a subsequence of any other frequent subsequence.
    * A frequent sequence is *closed* if it's not a subseqeuence of any other frequent sequence with the same support.

## 10.2 Mining Frequent Sequences
* For itemset mining, only need to consider combinations of items
* because order matters in sequences, need to consider all permutations.
* Sequence search space can be used in prefix search tree.
    * Root is an empty set, all children are symbols.
    * If a node is labeled with a sequence $s = s_1s_2...s_k$ at level k, then it's child would have to form of $s' = s_1s_2...s_ks_{k+1}$ at level k+1. Thus, s is a prefix of s', also known as an extension of s.
* Page 290 has a picture of the subsequence search space.
    * Greyed out nodes are sequences that do not satisfy the min support.
    * The picture has minsup of 3.
    * Technically, this search space is infinite, it includes all sequences of length zero or more using the symbols denoted in $\sum$.
    * Thus, if l is the length of hte longest sequence in the database, the worst case is to consider **ALL** sequences of length up to l, which means that size of the search space is bounded by $O(|\sum|^l)$ where there are a total of $|\sum|^k$ possible subsequences of length k 

### 10.2.1 Level-Wise Mining: GSP 
* Given a set of k-sized frequent sequences (equivalent to $C^k$), generate all possible candidates, or sequence "extensions" by extending by a single letter, at level k+1.
* Compute the support and prune, stops when frequent extensions don't exist.

In [1]:
def GSP(D, S, minsup):
    # C is a dictionary of sets - contains POSSIBLE candidates that are frequent
    # F is a dictionary of sets - contains candidates that are frequent
    F = {}
    C = {}
    # constructing C(1)
    C[1] = set()
    # construct root node
    root = Node(0)
    for s in S:
        # constructing a node to easily store support values, will automatically store 0 to support
        temp = Node(s)
        # adds s as a child of root
        root.children.add(temp)
        C[1].add(temp)
    # denotes the level
    k = 1
    while C[k] != set():
        F[k] = set()
        compute_support(C[k], D)
        for s in C[k]:
            if sup(s) >= minsup:
                F[k].add((s, sup(s)))
            else:
                C[k].remove(s)
        C[k+1] = extend_prefix_tree(C[k])
        k += 1
    return F    

In [4]:
def compute_support(C_k, D):
    for row in D:
        # r is each possible candidate that has length k
        for r in C_k:
            # if r contains s_i (row), r is a subsequence or a substring of s_i
            if r in row:
                r.sup += 1

In [5]:
def extend_prefix_tree(C_k):
    for r_a in C_k:
        # for each of the siblings of r_a who share the same parent
        for r_b in r_a.parent.children:
            # extend ra with the last symbol of rb because 
            r_ab = r_a + r_b[-1]
            # begin prunning process 
            # construct powerset of r_ab where length is |r_ab| - 1
            for r_c in powerset(r_ab, len(r_ab)-1):
                if r_c in C_k:
                    temp = Node(r_ab)
                    r_a.children.add(temp)
        if len(r_a.children) == 0:
            C_k.remove(r_a)
    return C_k

* E.x. of GSP Algo.
    * if r_a = AA, the children of r_a's parent, A, is AA and AG.
    * Thus, r_ab = AA + AA[-1] = AAA
    * The next iteration is AA + AG[-1] = AAG and etc.
* The complexity of GSP is $O(|\sum|^l)$ where l is the length of the longest frequency sequence.

### 10.2.2 Vertical Sequence Mining: SPADE
* SPADE uses vertical database.
* For each symbol record where the symbol occurs for each sequence in the form of $<i, pos(s)>$ where pos(s) is the set of positions in the sequence $\textbf{s}_i \in D$ where the symbol s occurs.  
* L(s) = the set of sequence-position tuples for symbol s, referred to as the poslist.
* Given a sequence of size k r, the poslist L(r) maintains the list of positions for the occurrences of the last occuring symbol in r in each database sequence $\textbf{s}_i$ provided that $r \subseteq s_i$
* Support 
    * E.x. given a database in this format:

In [7]:
import pandas as pd
pd.DataFrame({'id':['s1', 's2', 's3'], 'sequence':['CAGAAGT', 'TGACAG', 'GAAGT']})

,id,sequence
0,s1,CAGAAGT
1,s2,TGACAG
2,s3,GAAGT


if r = 'AA', L(r) = 

In [8]:
pd.DataFrame({'row':[1,2,3], 'end_pos':[(4,5), (5,), (3,)]})

,row,end_pos
0,1,"(4, 5)"
1,2,"(5,)"
2,3,"(3,)"


The symbol A occurs in all 3 strings at various positions, thus the corresponding node would look like the following and, since the symbol A appears in all 3 strings, the support is equal to 3. 

In [9]:
pd.DataFrame({'row':[1,2,3], 'end_post':[(2,4,5), (3,5), (2,3)]})

,row,end_post
0,1,"(2, 4, 5)"
1,2,"(3, 5)"
2,3,"(2, 3)"


* To compute support in Spade, perform sequential join
    * For two k length sequences $r_a$ and $r_b$ that share the same parent, or the same (k-1) length prefix, the idea is to perform sequential joins on the loslists (L(r)) to compute the support for the new (k+1) length candidate sequence $r_{ab} = r_a + r_b[k]$
    * <i, pos($r_a[k]$)> stands for given a sequence, the pos($r_a[k]$) is the position of the last character of the sequence $r_a$.
    * We first check if both $r_b[k]$ and $r_a[k]$ sequences exist in the same database sequence, $s_i$.
    * Next, for each position $p \in pos(r_b[k])$, which pretty much means checking back in level 1 and see where each letter occurs in each string in the database, we check wheter there exists a position $q \in pos(r_a[k])$ such that q <p
        * this ensures that when you add the last character from $r_b$, that the character comes after the last character of $r_a$.
        * that's important cause you want to append the last character or $r_b$ to ensure that order is perserved of the subsequence.
    * If q < p, then $r_b[k]$ occurs after the last position of $r_a$ and thus the value p is a **valid** occurrence of $r_{ab}$.
* The position list $L(r_{ab})$ is made up of all such valid occurrences.
* Only keep track of the last character of a valid sequence, since the prefix symbols provide no value.
* We denot the sequential join as $L(r_{ab}) = L(r_a) \cap L(r_b)$

In [1]:
# because of the recursive nature of this algo, need to declare things outside of the scope of the function
# pre-defined values, S = (summation) which represents 
# assume the database D is a list of strings, where the index of the element is the id of the string
# this is for singular symbols
def L(S, D, minsup):
    P = []
    for symbol in S:
        ind_lst = []
        for ind, s_i in enumerate(D):
            temp_lst = []
            for str_ind, character in enumerate(s_i):
                if symbol == character:
                    temp_lst.append(str_ind)
            # temp_lst = [2,4,5]
            ind_lst.append({ind, temp_lst})
        # accounts for minsup arg
        if len(ind_lst) >= minsup:
            P.append((symbol, ind_lst))
        
def seq_join(r_a, r_b, P):
    # r_a and r_b are lists of of (i, L(i)) where L(i) is a list of indices of where the corresponding 
    # symbol appears in the string.
    
    
    
F = 0
k = 0
# we assume P is defined already
P = None
def SPADE(P, minsup, F, k):
    # r_a is in the form of (symbol, L(symbol)) where L(symbol) is a list of dictionaries, each dictionary
    # corresponds to a string in the database (s_i) and the occurences of the symbol
    for r_a in P:
        F.append(r_a[0], len(r_a[1]))
        P_a = 0
        for r_b in P:
            r_ab = r_a + r_b[k]
            
        


IndentationError: expected an indented block (<ipython-input-1-fbaf9ad465a8>, line 26)